In [5]:
import sys
from pathlib import Path
# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path: sys.path.append(str(ROOT_DIR))

import pandas as pd
import strategies.ta as ta
import strategies.signals as sig
from  strategies.ta import TAData
from frame import Frame
from data.random_data import RandomOHLCV
from dataclasses import dataclass

periods = 400

# ---------------------------------------------
# --------------- DAILY FRAME -----------------
# ---------------------------------------------
import strategies.preset_strats as ps

spy = RandomOHLCV( freq='5 mins', start='2024',periods= periods).get_dataframe() #!supposd to be daily.  just used for testing 
etf = RandomOHLCV( freq='5 mins', start='2024',periods= periods).get_dataframe() #!supposd to be daily.  just used for testing
day = RandomOHLCV( freq='5 mins', start='2024',periods= periods, open_rng=(-0.35, 0.4)).get_dataframe() #!supposd to be daily.  just used for testing
f_day = Frame('TSLA', day, run_ta_on_load=True, rowHeights=[0.1, 0.1, 0.1, 0.1, 0.1, 0.5])
lookback = 100
ps.import_to_daily_df(f_day, spy, etf, RSIRow=3) #! test mansfield on real ccharts and compare to trading view
ps.require_ta_for_all(f_day)
ps.ma_ta(f_day, [50, 150, 200])
ps.volume_ta(f_day, ls='LONG', ma=10, scoreRow=4, lookBack=1)
ps.consolidation_ta(f_day, atrSpan=50, maSpan=50, lookBack=1, scoreRow=4)
ps.STRATEGY_daily_consolidation_bo(f_day, lookBack=lookback, scoreRow=5)
ps.STRATEGY_pullback_to_cons(f_day, ls='LONG', lookBack=lookback, scoreRow=5)
print('--- DAILY FRAME ---')
print(f_day.data.columns)
f_day.plot(height=1000, width=1800)
# display(f_day.data[['close','SigL_PBN_CON', 'FFILL_HP_hi_10']])

# # ---------------------------------------------
# # --------------- HTF FRAMEs -------------------
# hr1 = RandomOHLCV( freq='60 mins', start='2024',periods= periods).get_dataframe() #!supposd to be daily.  just used for testing
# m15 = RandomOHLCV( freq='15 mins', start='2024',periods= periods).get_dataframe()
# f_hr1 = Frame('TSLA', hr1, run_ta_on_load=True)
# f_m15 = Frame('TSLA', m15, run_ta_on_load=True)
# for f in [f_hr1, f_m15]:
#     require_ta_for_all(f)
#     ma_ta(f, [21, 50])
#     print('--- HTF FRAME ---')
#     f.plot()


# # ---------------------------------------------
# # --------------- MINUTE FRAMES ---------------
# # ---------------------------------------------
# m5  = RandomOHLCV( freq='5 mins', start='2024',periods= periods).get_dataframe()
# m2  = RandomOHLCV( freq='2 mins', start='2024',periods= periods).get_dataframe()
# m1  = RandomOHLCV( freq='1 min' , start='2024',periods= periods).get_dataframe()
# f_m5  = Frame('TSLA', m5,  run_ta_on_load=True)
# f_m2  = Frame('TSLA', m2,  run_ta_on_load=True)
# f_m1  = Frame('TSLA', m1,  run_ta_on_load=True)

# for f in [f_m5, f_m2, f_m1]:
#     require_ta_for_all(f)
#     ma_ta(f, [8, 21, 50], ma_col='close', row=1)
#     volume_ta(f, ls='LONG', ma=13, scoreRow=4)
#     consolidation_ta(f, atrSpan=50, maSpan=50, lookBack=1)
#     print('--- MINUTE FRAME ---')
#     f.plot()


f_day.backtest_setup(start=-100, end=-1, save_snapshots=True)
f_day.backtest_run(clear_snapshots=True)
f_day.plot(height=1000, width=1800)

Tested rows 94/99 - Date: 2024-01-02 08:45:00 (location 393)


In [ ]:
f_day.run_snapshots(0,50, plot=True, display_df=False, sleep_time=0.3, width=2000, height=1500)

In [46]:
image_path=r'C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\tests\test1.png'
f_day.chart.fig.write_image(image_path)

In [56]:

def run_until_found_nth_scores(min_score=0.7, target_scores=10, image_path=r'C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\tests'):
    """
    Run trading strategies until finding n number of valid scores above minimum threshold.
    Continues running even if errors occur in individual iterations.
    """
    import pandas as pd
    from datetime import datetime
    import os
    import traceback
    
    # Initialize logging dataframe with error column
    log_data = {
        'iteration': [],
        'datetime': [],
        'bo_score': [],
        'pb_score': [],
        'final_score': [],
        'valid_score': [],
        'image_saved': [],
        'error': []
    }
    
    periods = 400
    scores_found = 0
    count = 0
    
    while scores_found < target_scores:
        count += 1
        error_msg = None
        
        try:
            # Frame creation and strategy execution
            spy = RandomOHLCV(freq='5 mins', start='2024', periods=periods).get_dataframe()
            etf = RandomOHLCV(freq='5 mins', start='2024', periods=periods).get_dataframe()
            day = RandomOHLCV(freq='5 mins', start='2024', periods=periods, open_rng=(-0.35, 0.4)).get_dataframe()
            f_day = Frame('TSLA', day, run_ta_on_load=True, rowHeights=[0.1, 0.1, 0.1, 0.1, 0.1, 0.5])
            
            lookback = 300
            import_to_daily_df(f_day, spy, etf, RSIRow=3)
            require_ta_for_all(f_day)
            ma_ta(f_day, [50, 150, 200])
            volume_ta(f_day, ls='LONG', ma=10, scoreRow=4, lookBack=lookback)
            consolidation_ta(f_day, atrSpan=50, maSpan=50, lookBack=lookback, scoreRow=4)
            STRATEGY_daily_consolidation_bo(f_day, lookBack=lookback, scoreRow=5)
            STRATEGY_pullback_to_cons(f_day, lookBack=lookback, scoreRow=5)
            
            # Get max scores
            max_bo_score = f_day.data['PBX_BO_Score'].max()
            max_pb_score = f_day.data['PBX_PB_Score'].max()
            max_final_score = f_day.data['PBX_ALL_Scores'].max()
            
            # Check if score is valid
            is_valid = max_final_score >= min_score
            image_saved = False
            
            # Save image if valid score found
            if is_valid:
                scores_found += 1
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                image_name = f'valid_score_{timestamp}_score_{max_final_score:.2f}.png'
                full_image_path = os.path.join(image_path, image_name)
                
                try:
                    f_day.plot(height=1000, width=1800)
                    f_day.chart.fig.write_image(full_image_path)
                    image_saved = True
                except Exception as img_err:
                    error_msg = f"Image save error: {str(img_err)}"
                
                print(f"[VALID SCORE {scores_found}/{target_scores}] Iteration {count}: BO={max_bo_score:.2f}, PB={max_pb_score:.2f}, Final={max_final_score:.2f}")
            else:
                print(f"Iteration {count}: Final={max_final_score:.2f} ({scores_found}/{target_scores} found)")

        except Exception as e:
            error_msg = f"Iteration error: {str(e)}"
            print(f"Error in iteration {count}: {error_msg}")
            # Set default values for error cases
            max_bo_score = max_pb_score = max_final_score = 0
            is_valid = image_saved = False
        
        # Log iteration data (including error cases)
        log_data['iteration'].append(count)
        log_data['datetime'].append(datetime.now())
        log_data['bo_score'].append(max_bo_score)
        log_data['pb_score'].append(max_pb_score)
        log_data['final_score'].append(max_final_score)
        log_data['valid_score'].append(is_valid)
        log_data['image_saved'].append(image_saved)
        log_data['error'].append(error_msg)
    
    # Create final log dataframe
    log_df = pd.DataFrame(log_data)
    
    # Print summary statistics
    print("\n=== Final Summary ===")
    print(f"Total iterations: {count}")
    print(f"Success rate: {(scores_found/count)*100:.2f}%")
    print(f"Average final score: {log_df['final_score'].mean():.2f}")
    print(f"Max final score: {log_df['final_score'].max():.2f}")
    print(f"Total errors: {log_df['error'].notna().sum()}")
    
    return log_df

In [61]:
# Run the function with custom parameters
log_df = run_until_found_nth_scores(min_score=80, target_scores=20)

# View the results
print("\nDetailed log of valid scores:")
print(log_df[log_df['valid_score']].to_string())

Iteration 1: Final=66.66 (0/20 found)


KeyboardInterrupt: 

In [4]:
f_day.backtest_setup(start=-10, end=-1, save_snapshots=True)
f_day.backtest_run(clear_snapshots=True)

f_day.run_snapshots(0,50, plot=True, display_df=False, sleep_time=0.1, width=2000, height=1500)


Viewing snapshot 10/10 - Date: 2024-01-02 09:10:00 (location 398)


In [5]:

f_day.get_snapshot(5, plot=True, width=2000, height=1500)

Viewing snapshot 6/10 - Date: 2024-01-02 08:50:00 (location 394)


,open,high,low,close,volume,SPY_close,ETF_close,MRSI_100_SPY,MRSI_100_ETF,ATR_50,...,Stgy_PB_1-PB_Scores_Av,Stgy_PB_1-RtcPiv > 50,Stgy_PB_1-PBnrCons > 50,Stgy_PB_step1,Stgy_PB_total,PBX_BO_Score,PBX_PB_Score,PBX_ALL_Scores,CONS_UPPER_2,CONS_LOWER_2
date,,,,,,,,,,,,,,,,,,,,,
2024-01-01 00:00:00,99.98,100.52,99.83,100.23,100,99.75,100.17,0.000000,0.000000,0.690000,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:05:00,100.63,101.10,100.34,100.86,100,99.30,99.60,0.000000,0.000000,0.725000,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:10:00,100.74,101.37,100.49,101.09,39,98.86,99.55,-0.707107,-0.707107,0.776667,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:15:00,101.11,101.19,100.58,100.86,40,98.97,99.03,-1.108955,-0.560193,0.735000,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:20:00,100.71,100.88,100.63,100.71,35,99.13,98.94,-0.821963,-0.904050,0.638000,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-02 08:30:00,105.57,105.82,105.52,105.59,5,90.18,97.95,1.136960,-0.054127,0.504600,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2024-01-02 08:35:00,105.26,105.50,105.03,105.18,5,90.21,98.25,-0.893746,-1.544377,0.503600,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2024-01-02 08:40:00,104.95,105.07,104.51,104.79,7,90.29,98.23,-0.963563,-0.767004,0.506000,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [5]:
f.data# .tail(100)
m5.tail(50)
f.backtest_data.tail(50)

,open,high,low,close,volume,SPY_close,MA_SP_5,ATR_50,MA_cl_50,MA_cl_100,...,Sup_1,Sup_1_Upper,Sup_1_Lower,Sup_2,Sup_2_Upper,Sup_2_Lower,CONS_UPPER_1,CONS_LOWER_1,CONS_UPPER_2,CONS_LOWER_2
date,,,,,,,,,,,,,,,,,,,,,
2024-01-01 05:14:00,98.84,99.29,98.75,99.23,56,97.77,97.770,0.5142,99.1120,98.8599,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
2024-01-01 05:15:00,99.56,100.07,99.31,99.79,73,97.85,97.786,0.5200,99.1644,98.8588,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
2024-01-01 05:16:00,99.48,99.60,99.21,99.52,43,97.85,97.802,0.5168,99.2100,98.8602,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
2024-01-01 05:17:00,99.37,99.49,99.32,99.44,42,97.85,97.818,0.5142,99.2466,98.8605,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
2024-01-01 05:18:00,99.72,99.94,99.63,99.78,29,97.85,97.834,0.5100,99.2936,98.8625,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
2024-01-01 05:19:00,99.90,99.99,99.74,99.98,15,97.85,97.850,0.5072,99.3494,98.8722,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
2024-01-01 05:20:00,99.58,99.76,99.13,99.24,17,97.37,97.754,0.5092,99.3790,98.8700,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
2024-01-01 05:21:00,98.89,99.22,98.75,99.02,10,97.37,97.658,0.5116,99.3980,98.8674,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
2024-01-01 05:22:00,98.71,98.73,98.37,98.41,6,97.37,97.562,0.5030,99.3924,98.8619,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
